## 1- Importing Packages

In [1]:
#packages for analysis
import pandas as pd
import numpy as np
import cv2 
from glob import glob

#packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns

#allows charts to appear in the notebook 
%matplotlib inline

#import OS to share the project dir
import os
os.chdir('H:\Barq')

#ignoring warnings
import warnings
warnings.filterwarnings('ignore')


## 2- Define Constants

In [2]:
#define our paths
BASE_DATA_FOLDER = "Plants"
TRAIN_DATA_FOLDER = os.path.join(BASE_DATA_FOLDER, "train")
RESULT_DATA_FOLDER=os.path.join(BASE_DATA_FOLDER, "result")

## 3-Reading Unsegmented Data

In [3]:
images_per_class = {}
labels=['Black-grass','Charlock','Cleavers','Common Chickweed','Common wheat'
        ,'Fat Hen','Loose Silky-bent','Maize','Scentless Mayweed'
        ,'Shepherds Purse','Small-flowered Cranesbill','Sugar beet']
# get all classes name
for class_folder_name in os.listdir(TRAIN_DATA_FOLDER):
    class_folder_path = os.path.join(TRAIN_DATA_FOLDER, class_folder_name)
    class_label = class_folder_name
    images_per_class[class_label] = []
    print("reading data from class:",class_label)
    #get the images in each class
    for image_path in glob(os.path.join(class_folder_path, "*.png")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        images_per_class[class_label].append(image)

reading data from class: Black-grass
reading data from class: Charlock
reading data from class: Cleavers
reading data from class: Common Chickweed
reading data from class: Common wheat
reading data from class: Fat Hen
reading data from class: Loose Silky-bent
reading data from class: Maize
reading data from class: Scentless Mayweed
reading data from class: Shepherds Purse
reading data from class: Small-flowered Cranesbill
reading data from class: Sugar beet


## 4-Segmentation

In [4]:
#apply gaussian 
def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [ ]:
#convert to hsv and do segmentation
print ("Read My Data") 
x_train = [] 
y_train = [] 
for j in range(12):
    count=0
    for img in images_per_class[labels[j]] :
            count=count+1
            ## convert to hsv 
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            
            ## mask of green (36,0,0) ~ (70, 255,255) #(25, 0, 10), (70, 255,255)
            #best (25, 40, 30), (95, 255,255)
            mask = cv2.inRange(hsv, (25, 40, 50), (75, 255, 255))
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            
            ## slice the green
            imask = mask>0
            green = np.zeros_like(img, np.uint8)
            green[imask] = img[imask]
            green=sharpen_image(green)
            x_train.append(np.array(img))
            y_train.append(labels[j])
            
            ## save my new image
            image_name = labels[j]+str(count)+".png"
           # imshow(green)
            #cv2.imwrite(imageName, green)
            cv2.imwrite(os.path.join(RESULT_DATA_FOLDER ,image_name), green)
            
print("Done")

Read My Data


# Reading Segmented Data

In [8]:
seg_images=[]
for image_path in glob(os.path.join(RESULT_DATA_FOLDER, "*.png")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        seg_images.append(image)

## Applying Min Filter